In [3]:
# install fastparquet via system call
# !pip3 install fastparquet
# !pip3 install statsmodels 
# !pip3 install matplotlib
import fastparquet 
import pandas as pd, os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp
import statsmodels.formula.api as smf

In [2]:
# set worknig directory to be wherever your data is
try: 
    os.chdir('/Volumes/evo/Dropbox (Personal)/teaching_2025_scratch')
except:
    print("need to point to the right directory")
    

In [ ]:
df = pd.read_parquet('hw2_mfin7037_data.parquet')
# start the data in 1994, before which there's some weird stuff
df = df.query("date>='1994-01-01'")
# remember to fitler the data
df.head()

In [4]:
def produce_table(input_data, subsetting=lambda df: df):
    """
    Combines portfolio data from 'bins' and 'pnl', computes summary statistics
    (mean returns multiplied by 100 and one-sample t-test statistics) for each portfolio,
    and returns a pivoted table for portfolios 1..10 and a long-short portfolio.
    
    Parameters:
        input_data (dict): Dictionary with keys:
            - 'bins': a DataFrame with columns ['date','bin','ew','vw',
                      'ew_intraday','vw_intraday','ew_overnight','vw_overnight']
            - 'pnl': a DataFrame with columns ['date','ew','vw',
                      'ew_intraday','vw_intraday','ew_overnight','vw_overnight']
                      (this will be used to compute the long-short portfolio)
        subsetting (function): A function to subset/modify the combined DataFrame.
                               Defaults to the identity function.
    
    Returns:
        pd.DataFrame: A table where each row corresponds to a return measure (e.g., EW, VW, etc.)
                      with the first row showing the mean (×100, rounded to 3 decimals) and the second
                      row showing the t-statistic (in parentheses). The long-short portfolio is labeled "10-1".
    """
    
    # Extract the required columns from each DataFrame.
    bins_df = input_data['bins'][['date', 'bin', 'ew', 'vw', 
                                  'ew_intraday', 'vw_intraday', 
                                  'ew_overnight', 'vw_overnight']].copy()
    pnl_df = input_data['pnl'][['date', 'ew', 'vw', 
                                'ew_intraday', 'vw_intraday', 
                                'ew_overnight', 'vw_overnight']].copy()
    # Designate the pnl data as portfolio "11" (which later will be renamed to "10-1")
    pnl_df['bin'] = 11

    # Combine the two datasets and apply any subsetting
    combined = pd.concat([bins_df, pnl_df], ignore_index=True)
    combined = subsetting(combined)

    # List of return measure columns
    cols = ['ew', 'vw', 'ew_intraday', 'vw_intraday', 'ew_overnight', 'vw_overnight']

    # Function to compute mean multiplied by 100 and rounded to 3 decimals
    def meanna(x):
        return round(x.mean() * 100, 3)

    # Compute group-wise means by portfolio (bin)
    # 按投资组合bin分组计算收益的均值
    s1 = combined.groupby('bin')[cols].agg(meanna).reset_index()
    s1_melt = s1.melt(id_vars='bin', var_name='variable', value_name='value')
    s1_pivot = s1_melt.pivot(index='variable', columns='bin', values='value').reset_index()
    s1_pivot['order'] = 1

    # Function to compute one-sample t-test statistic (null: mean=0), rounded to 3 decimals and wrapped in parentheses.
    # 检验样本均值是否显著不同于0(零假设H_0:μ=0) 
    def ttesting(x):
        x = x.dropna()
        if len(x) == 0:
            return None
        stat, _ = ttest_1samp(x, popmean=0) # Compute t-statistic (stat), _ is the p-value (ignored)
        return f"({round(stat, 3)})"

    # Compute group-wise t-statistics by portfolio (bin)
    s2 = combined.groupby('bin')[cols].agg(ttesting).reset_index()
    s2_melt = s2.melt(id_vars='bin', var_name='variable', value_name='value')
    s2_pivot = s2_melt.pivot(index='variable', columns='bin', values='value').reset_index()
    s2_pivot['order'] = 2

    # Combine the mean and t-statistic tables
    table = pd.concat([s1_pivot, s2_pivot], ignore_index=True)
    table = table.sort_values(by=['variable', 'order'])
    # For rows with t-statistics (order 2), clear the 'variable' name
    table.loc[table['order'] == 2, 'variable'] = ''
    table = table.drop(columns=['order'])

    # Format the variable names: uppercase and replace underscores with spaces
    table['variable'] = table['variable'].str.upper().str.replace('_', ' ', regex=False)
    table = table.rename(columns={'variable': 'Portfolio'})

    # Rename portfolio 11 to "10-1" (i.e. the long-short portfolio)
    if 11 in table.columns:
        table = table.rename(columns={11: '10-1'})
    
    # Ensure that only portfolios 1 through 10 and the long-short ("10-1") remain.
    valid_portfolios = list(range(1, 11)) + ['10-1']
    cols_to_keep = ['Portfolio'] + [col for col in table.columns if col in valid_portfolios]
    table = table[cols_to_keep]

    return table


# --- Helper: Apply quantiles within each date group ---
def apply_quantiles(df, col, bins=10):
    """
    Assigns a quantile-based bin (1,...,bins) for each value in `col`
    within each date group.
    """
    def quantile_bin(s):
        # Use pd.qcut to get quantile bins; if duplicate edges occur, use rank instead.
        try:
            return pd.qcut(s, q=bins, labels=False, duplicates="drop") + 1
        except Exception:
            return np.ceil(s.rank(method='average') / len(s) * bins)
    return df.groupby('date')[col].transform(quantile_bin)

In [5]:
def routine_pnl(dta, plot=False):
    """
    Computes portfolio returns (EW/VW) by bin, optionally creates two plots 
    (mean returns by bin and cumulative pnl), runs a series of OLS regressions, 
    and returns a dictionary with outputs.
    
    Assumes dta contains columns: date, bin, ret, mcap_lag1, 
    intraday_ret_month, overnight_ret_month.
    
    Parameters:
      dta   : pd.DataFrame
              Input data.
      plot  : bool, default False
              If True, plots are created; if False, they are suppressed.
    
    Returns:
      dict with keys:
          'p1'             : Axes for decile plot (or None if plot==False)
          'pnl_curve'      : Axes for cumulative pnl plot (or None if plot==False)
          'factor_loadings': Dictionary of regression results
          'pnl'            : DataFrame of pnl calculations
          'bins'           : DataFrame of computed bin returns
    """
    # Ensure date is datetime
    dta = dta.copy()
    dta['date'] = pd.to_datetime(dta['date'])
    
    # Helper: Weighted mean
    def weighted_mean(x, w):
        return np.average(x, weights=w)
    
    # Compute portfolio returns by date and bin
    bins = (dta.groupby(['date', 'bin'])
            .apply(lambda g: pd.Series({
                'ew': g['ret'].mean(),
                'vw': weighted_mean(g['ret'], g['mcap_lag1']),
                'ew_intraday': g['intraday_ret_month'].mean() if 'intraday_ret_month' in g.columns else np.nan,
                'vw_intraday': weighted_mean(g['intraday_ret_month'], g['mcap_lag1']) if 'intraday_ret_month' in g.columns else np.nan,
                'ew_overnight': g['overnight_ret_month'].mean() if 'overnight_ret_month' in g.columns else np.nan,
                'vw_overnight': weighted_mean(g['overnight_ret_month'], g['mcap_lag1']) if 'overnight_ret_month' in g.columns else np.nan,
            }))
            .reset_index())
    
    # Create decile plot (collapsed across dates) if plotting is requested
    if plot:
        bins_collapsed = bins.groupby('bin')[['ew','vw']].mean().reset_index()
        fig1, ax1 = plt.subplots()
        ax1.plot(bins_collapsed['bin'], bins_collapsed['ew'], label='EW', marker='')
        ax1.plot(bins_collapsed['bin'], bins_collapsed['vw'], label='VW', marker='')
        ax1.set_xlabel('Bin')
        ax1.set_ylabel('Return')
        ax1.set_title('Mean Portfolio Returns by Bin')
        ax1.legend()
        plt.tight_layout()
    else:
        ax1 = None
    
    # Compute long-short pnl using the extreme bins (assume bin 1 and max(bin))
    # 筛选出 bin 列为 1 和 max_bin 的行，表示最低和最高分组的数据。这些数据代表long-short策略中需要操作的分组。
    max_bin = bins['bin'].max()
    selected = bins[bins['bin'].isin([1, max_bin])].copy()
    selected['weight'] = selected['bin'].apply(lambda x: -1 if x == 1 else 1)
    pnl = (selected.groupby('date')
           .apply(lambda g: pd.Series({
               'ew': (g['weight'] * g['ew']).sum(),
               'vw': (g['weight'] * g['vw']).sum(),
               'ew_intraday': (g['weight'] * g['ew_intraday']).sum() if 'ew_intraday' in g.columns else np.nan,
               'vw_intraday': (g['weight'] * g['vw_intraday']).sum() if 'vw_intraday' in g.columns else np.nan,
               'ew_overnight': (g['weight'] * g['ew_overnight']).sum() if 'ew_overnight' in g.columns else np.nan,
               'vw_overnight': (g['weight'] * g['vw_overnight']).sum() if 'vw_overnight' in g.columns else np.nan,
           })).reset_index())
    pnl = pnl.sort_values('date')
    pnl['cumvw'] = (1 + pnl['vw']).cumprod() - 1
    pnl['cumew'] = (1 + pnl['ew']).cumprod() - 1
    
    # Create cumulative pnl plot if plotting is requested
    if plot:
        fig2, ax2 = plt.subplots()
        ax2.plot(pnl['date'], pnl['cumew'], label='EW')
        ax2.plot(pnl['date'], pnl['cumvw'], label='VW')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Cumulative Return')
        ax2.set_title('Cumulative PnL Over Time')
        ax2.legend()
        plt.tight_layout()
    else:
        ax2 = None
    
    # Minimal regressions for demonstration, not done tho
    formulas = [
        'ew ~ 1',  
        'vw ~ 1',
        'ew ~ 0',  # dummy formula if needed
    ]
    regs = {}
    for bin_val in sorted(bins['bin'].unique()):
        data_bin = bins[bins['bin'] == bin_val]
        for fml in formulas:
            key = f'bin {bin_val} fml {fml}'
            try:
                model = smf.ols(formula=fml, data=data_bin).fit()
            except Exception as e:
                model = None
                print(f"Regression failed for bin {bin_val} with formula '{fml}': {e}")
            regs[key] = model
            
    # Return outputs in a dictionary
    return {
        'p1': ax1,                # decile plot (or None)
        'pnl_curve': ax2,         # cumulative pnl plot (or None)
        'factor_loadings': regs,  # regression results
        'pnl': pnl,               # pnl table
        'bins': bins              # bins table (with computed returns)
    }


In [6]:
# --- Strategy 1 ---
# (Following the R code, first a ranking step is done then overwritten by quantiles on mom_intraday)
res2_strategy1 = df.copy()
# Here we assign bin using mom_intraday quantiles (10 bins)
res2_strategy1['bin'] = apply_quantiles(res2_strategy1, 'mom_overnight', bins=10)
subset1 = res2_strategy1.dropna(subset=['bin', 'mcap_lag1', 'ret'])
strat1 = routine_pnl(subset1, plot=True)
table1 = produce_table({'bins': strat1['bins'], 'pnl': strat1['pnl']})
print("=== Strategy 1 Table ===")
# print the table
#  order by index and plot table
table1



# Question 1

### 1.1 Information Events Happening: Intraday vs. Overnight

#### (1) Information Events During Market Hours
- **News Announcements**:  
  The company may make some important news or announcements during trading hours, such as management changes, major acquisitions, or business restructuring.

- **Market Dynamics**:  
  Breaking events that may occur during trading hours, such as natural disasters, geopolitical conflicts, or other news that affects the market.

#### (2) Information Events Outside of Market Hours
- **Earnings Announcements**:  
  Most companies choose to announce earnings outside of market hours (before or after the bell).

- **Macroeconomic Data**:  
  Most macro data is usually released before the market opens.

- **Policy Announcements**:  
  Central bank policy announcements may be made outside of trading hours, especially in the event of major policy adjustments.


### 1.2 Timing of the Company's Earnings Announcement
Earnings announcements are usually issued outside of market hours to reduce market volatility and give investors more time to interpret information. Liu J, Hope O-K, and Hu D (2023) show that abnormal overnight returns on earnings announcement days exhibit strong predictability for future stock returns, consistent with the overnight returns containing value-relevant signals.  

### 1.3 Macro Announcements

#### (1) Time of Release  
Usually, macro announcements are released before trading hours. For example, NFP is published on the first Friday of each month, at 8:30 a.m. EST. Other macroeconomic data such as GDP, CPI, PPI, etc. are also released before the market opens.

#### (2) Why Would Macro Announcements Matter for Individual Stocks?  

##### a. **Influence Investor Sentiment**  
Macro announcements reflect the overall health of the economy and can significantly influence market sentiment.  
- **Positive data**: For example, strong non-farm payrolls data can indicate a booming economy and boost investor confidence in cyclical sectors (e.g. consumer, manufacturing), which can drive related stocks higher.  
- **Negative data**: Conversely, weak data can raise fears of a recession, leading to a general decline in the stock market.

##### b. **Influence Interest Rate Expectations and Monetary Policy**  
Macro data can change market expectations of central bank policy (e.g. rate hikes or cuts).  
- **Expectations of interest rate hike**: Strong economic data can prompt central banks to raise interest rates, leading to valuation pressure on growth stocks.  
- **Expectations of interest rate cut**: Weak economic data can trigger expectations of easing, favoring highly leveraged or interest rate sensitive sectors.

##### c. **Direct Correlation Between Industries and Individual Stocks**
Certain industries are more sensitive to macro data, resulting in a direct impact on the relevant individual stocks:
- **Non-farm payroll data**: Affects the profitability of employment-intensive industries (e.g. retail, services).  
- **Inflation data (CPI/PPI)**: Affects consumer goods and materials industries, which can lead to changes in cost structure.  
- **Interest rate data**: Affects the profitability of financial sectors (e.g. banks, insurance companies) as interest rates are directly related to their business model.

# Question 2

### What is the correlaiton of intraday, overnight momentum and regular momentum?

In [7]:
# Calculate the correlation matrix for the specified columns
correlation_matrix = df[['mom_intraday', 'mom_overnight', 'mom']].corr()
correlation_matrix

# Question 3

In [8]:
data = pd.read_parquet('crsp_202401.dsf.parquet')
data.dropna(inplace=True)
data = data[['permno', 'date', 'ret', 'prc', 'openprc', 'retx']]
# 'retx': return excluding dividends
data.sort_values(['permno', 'date'], inplace=True)

In [7]:
# 储存data为parquet
data.to_parquet('data_Q3.parquet') 

NameError: name 'data' is not defined

In [8]:
data_Q3 = pd.read_parquet('data_Q3.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'data_Q3.parquet'

In [ ]:
data_Q3['intraday_ret'] = data_Q3['prc'] / data_Q3['openprc'] - 1
data_Q3['overnight_ret'] = (1 + data_Q3['ret']) / (1 + data_Q3['intraday_ret']) - 1
data_Q3

,permno,date,ret,prc,openprc,retx,intraday_ret,overnight_ret
518093,10001,1992-06-16,0.000000,12.00000,12.00000,0.000000,0.000000,0.000000e+00
518094,10001,1992-06-17,-0.020833,11.75000,11.75000,-0.020833,0.000000,-2.083300e-02
518095,10001,1992-06-18,0.000000,11.75000,11.75000,0.000000,0.000000,0.000000e+00
518096,10001,1992-06-19,0.042553,12.25000,11.75000,0.042553,0.042553,-1.836735e-07
518099,10001,1992-06-24,-0.010309,12.00000,12.00000,-0.010309,0.000000,-1.030900e-02
...,...,...,...,...,...,...,...,...
75021095,93436,2023-12-22,-0.007701,252.53999,256.76001,-0.007701,-0.016436,8.880618e-03
75021096,93436,2023-12-26,0.016116,256.60999,254.49001,0.016116,0.008330,7.721371e-03
75021097,93436,2023-12-27,0.018822,261.44000,258.35001,0.018822,0.011960,6.780423e-03
75021098,93436,2023-12-28,-0.031594,253.17999,263.66000,-0.031594,-0.039748,8.491729e-03


In [ ]:
window = 12

# 定义滚动累积乘积函数
def rolling_prod(arr, window):
    return arr.rolling(window=window).apply(lambda x: np.prod(x + 1) - 1, raw=True)

# 计算日间动量
data_Q3["intraday_momentum"] = data_Q3.groupby("permno")["intraday_ret"].transform(
    lambda x: rolling_prod(x, window)
)

# 计算隔夜动量
data_Q3["overnight_momentum"] = data_Q3.groupby("permno")["overnight_ret"].transform(
    lambda x: rolling_prod(x, window)
)

# 滞后处理（例如滞后 2 天）
data_Q3["intraday_momentum"] = data_Q3.groupby("permno")["intraday_momentum"].shift(2)
data_Q3["overnight_momentum"] = data_Q3.groupby("permno")["overnight_momentum"].shift(2)

# 删除缺失值
data_Q3 = data_Q3.dropna().reset_index(drop=True)

# 检查结果
data_Q3[["permno", "date", "intraday_momentum", "overnight_momentum"]]

          permno        date  intraday_momentum  overnight_momentum  \
0          10001  1992-07-23           0.132697           -0.109850   
1          10001  1992-07-24           0.132697           -0.082319   
2          10001  1992-07-27           0.132697           -0.033808   
3          10001  1992-07-28           0.132697           -0.033808   
4          10001  1992-07-29           0.086464           -0.033808   
...          ...         ...                ...                 ...   
26989810   93436  2023-12-22           0.048968            0.000099   
26989811   93436  2023-12-26           0.038263            0.026814   
26989812   93436  2023-12-27           0.036344            0.018022   
26989813   93436  2023-12-28           0.040282            0.016624   
26989814   93436  2023-12-29           0.037312            0.033612   

          total_momentum  
0               0.008271  
1               0.039454  
2               0.094402  
3               0.094402  
4           

In [19]:
data_Q3[["permno", "date", "intraday_momentum", "overnight_momentum", "total_momentum"]]

,permno,date,intraday_momentum,overnight_momentum,total_momentum
0,10001,1992-07-23,0.132697,-0.109850,0.008271
1,10001,1992-07-24,0.132697,-0.082319,0.039454
2,10001,1992-07-27,0.132697,-0.033808,0.094402
3,10001,1992-07-28,0.132697,-0.033808,0.094402
4,10001,1992-07-29,0.086464,-0.033808,0.049733
...,...,...,...,...,...
26989810,93436,2023-12-22,0.048968,0.000099,0.049071
26989811,93436,2023-12-26,0.038263,0.026814,0.066103
26989812,93436,2023-12-27,0.036344,0.018022,0.055021
26989813,93436,2023-12-28,0.040282,0.016624,0.057576


In [4]:
a = 10

In [3]:
d = 10
c = 19
dd = 23

In [4]:
a1 = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', skiprows=3)
a1

,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF
0,196307,-0.39,-0.41,-0.97,0.68,-1.18,0.27
1,196308,5.07,-0.80,1.80,0.36,-0.35,0.25
2,196309,-1.57,-0.52,0.13,-0.71,0.29,0.27
3,196310,2.53,-1.39,-0.10,2.80,-2.01,0.29
4,196311,-0.85,-0.88,1.75,-0.51,2.24,0.27
...,...,...,...,...,...,...,...
796,2020,23.66,3.02,-46.10,-4.93,-12.10,0.45
797,2021,23.57,-1.05,25.39,26.74,12.16,0.04
798,2022,-21.58,-1.69,25.97,6.67,22.54,1.43
799,2023,21.69,-5.99,-13.70,6.30,-20.93,4.95


In [4]:
a2 = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', skiprows=3)
a2

,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF
0,196307,-0.39,-0.41,-0.97,0.68,-1.18,0.27
1,196308,5.07,-0.80,1.80,0.36,-0.35,0.25
2,196309,-1.57,-0.52,0.13,-0.71,0.29,0.27
3,196310,2.53,-1.39,-0.10,2.80,-2.01,0.29
4,196311,-0.85,-0.88,1.75,-0.51,2.24,0.27
...,...,...,...,...,...,...,...
796,2020,23.66,3.02,-46.10,-4.93,-12.10,0.45
797,2021,23.57,-1.05,25.39,26.74,12.16,0.04
798,2022,-21.58,-1.69,25.97,6.67,22.54,1.43
799,2023,21.69,-5.99,-13.70,6.30,-20.93,4.95


In [6]:
a1.head()


,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF
0,196307,-0.39,-0.41,-0.97,0.68,-1.18,0.27
1,196308,5.07,-0.80,1.80,0.36,-0.35,0.25
2,196309,-1.57,-0.52,0.13,-0.71,0.29,0.27
3,196310,2.53,-1.39,-0.10,2.80,-2.01,0.29
4,196311,-0.85,-0.88,1.75,-0.51,2.24,0.27
